In [128]:
import requests
import pandas as pd
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from bs4 import BeautifulSoup as BS
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import StaleElementReferenceException
import configparser
import re
from datetime import timedelta, datetime
import time
import sys
sys.path.insert(0, '/Users/irarickman/StravaPythonClient')
import swagger_client
from swagger_client.rest import ApiException
from pprint import pprint

In [20]:
w='https://app.whoop.com/login'

In [44]:
browser=webdriver.Chrome('/usr/local/bin/chromedriver')
browser.get(w)

In [45]:
username=browser.find_element_by_name('username')
password=browser.find_element_by_name('password')
username.send_keys('irarickman@gmail.com')
password.send_keys('triathlon92')
submit=browser.find_element_by_name('form')
submit.click()

In [23]:
strain_click=browser.find_elements_by_class_name('score')[0]
strain_click.click()

In [23]:
strain_amt=browser.find_element_by_class_name("workout-strain")
strain_amt.get_attribute('innerHTML')
strain_amt.find_element_by_tag_name("strong").text

'16.2'

In [24]:
buttons=browser.find_elements_by_tag_name('button')
#[x for x in [span.find_elements_by_tag_name('span')  
[x.text for item in [x for x in [span.find_elements_by_tag_name('span') for span in [b for b in buttons if b.get_attribute("ng-click")=="click($event, activity.id)"]]] for x in item]


['Running', '16.2']

In [25]:
for x in buttons:
    if x.get_attribute("ng-click")=="click($event, activity.id)":
        spans=x.find_elements_by_tag_name("span")
        print([x.text for x in spans], x)
    else:
        print("nothing", x)

nothing <selenium.webdriver.remote.webelement.WebElement (session="08fde42d8d6815291fceb7d2092c9f08", element="0.7561134548339705-10")>
nothing <selenium.webdriver.remote.webelement.WebElement (session="08fde42d8d6815291fceb7d2092c9f08", element="0.7561134548339705-11")>
['Running', '16.2'] <selenium.webdriver.remote.webelement.WebElement (session="08fde42d8d6815291fceb7d2092c9f08", element="0.7561134548339705-12")>
nothing <selenium.webdriver.remote.webelement.WebElement (session="08fde42d8d6815291fceb7d2092c9f08", element="0.7561134548339705-13")>
nothing <selenium.webdriver.remote.webelement.WebElement (session="08fde42d8d6815291fceb7d2092c9f08", element="0.7561134548339705-14")>
nothing <selenium.webdriver.remote.webelement.WebElement (session="08fde42d8d6815291fceb7d2092c9f08", element="0.7561134548339705-15")>
nothing <selenium.webdriver.remote.webelement.WebElement (session="08fde42d8d6815291fceb7d2092c9f08", element="0.7561134548339705-16")>
nothing <selenium.webdriver.remote.w

In [24]:
def go_to_strain():
    work=0
    try:
        strain_click=browser.find_elements_by_class_name('score')[0]
        strain_click.click()
        work=1
    except StaleElementReferenceException:
        work=0

In [25]:
## goal function is to look for the activity - if it doesn't exist, then look for "no activity" 
## store the result, if the old activity strain is the same as the new

def get_activity_strain(old_strain,old_date):
    work=0
    while work==0:
        try:
            buttons=browser.find_elements_by_tag_name('button')
            new_strain=[x.text for item in [x for x in [span.find_elements_by_tag_name('span') for span in [b for b in buttons if b.get_attribute("ng-click")=="click($event, activity.id)"]]] for x in item]
            if len(new_strain)==0:
                new_strains=["null"]*6
            elif len(new_strain)<7:
                new_strains=new_strain+['null']*(6-len(new_strain))
            else:
                new_strains=new_strain
            if old_strain!=new_strains:
                return new_strains
                work=1
            elif old_date!=get_date(old_date):
                return new_strains
                work=1
        except StaleElementReferencException:
            work=0

In [26]:
def get_date(old_date):
    work=0
    while work==0:
        try:
            new_day=browser.find_element_by_class_name('datepicker--label')
            new_date=new_day.text
            if new_date!=old_date:
                return [new_date]
                work=1
        except StaleElementReferenceException:
            work=0

In [27]:
def get_scores(old_scores):
    work=0
    while work==0:
        try:
            new_scores=browser.find_elements_by_class_name('score')
            new_score_list=[x.text for x in new_scores]
            if new_score_list!=old_scores:
                return new_score_list
                work=1
        except StaleElementReferenceException:
            work=0
            

In [28]:
def get_button():
    work=0
    while work==0:
        try: 
            back=browser.find_element_by_class_name('datepicker--prev')
            back.click()
            work=1
        except StaleElementReferenceException:
            work=0

In [38]:
num=1
(get_date('today')[0].strip().lower()!='wed, may 30th' and num<300)

False

In [46]:
whoop=pd.DataFrame(columns=['strain','recovery','sleep_perf','sleep','rec_sleep','date', 
                            'activity_1','activity_1_score','activity_2','activity_2_score',
                           'activity_3','activity_3_score'])
num=0
old_date='today'
old_scores=['none']
old_strain=['null']
go_to_strain()
new_date=get_date(old_date)
while (new_date[0].strip().lower()!='wed, may 30th' and num<300):
    new_date=get_date(old_date)
    new_score=get_scores(old_scores)
    strains=get_activity_strain(old_strain,old_date)
    old_date=new_date
    old_scores=new_score
    old_strain=strains
    whoop.loc[len(whoop)]=new_score[:5]+new_date+strains
    get_button()
    num+=1

In [48]:
whoop.to_csv('../Data/whoop_data.csv',index=False)